## Analiza komunikacji miejskiej we Wrocławiu


**Tworzenie sieci**

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import networkx as nx
#df = pd.read_xml("0016.xml")

In [2]:
tree = ET.parse('data/XML-rozkladyjazdy/0016/0016.xml')
root = tree.getroot()

# lista słowników przechowujących wartości z elementów <czasy>
czasy_list = []
nazwa_lini = 16

# # iteruj po elementach XML i wyodrębnij wartości
# for linia in root.findall('linia'):
#     for wariant in linia.findall('wariant'):
#         for przystanek in wariant.findall('przystanek'):
#             for czasy in przystanek.find('czasy'):
#                 for przystanek_czas in czasy.findall('przystanek'):
#                         czasy_list.append({
#                             'nazwa': przystanek_czas.get('nazwa'),
#                             })

linia = root.find('linia')
wariant = linia.find('wariant')
przystanek = wariant.find('przystanek')
czasy = przystanek.find('czasy')
for przystanek_czas in czasy.findall('przystanek'):
    czasy_list.append({
                        'nazwa': przystanek_czas.get('nazwa'),
                         'czas': przystanek_czas.get('czas')})
df = pd.DataFrame(czasy_list)
df

# utwórz DataFrame z listy słowników
df = pd.DataFrame(czasy_list)

In [3]:
df
df['czas'] = df['czas'].fillna(0).astype(int)
df
#df = df.drop_duplicates(keep='first')

,nazwa,czas
0,ZOO,0
1,Hala Stulecia,1
2,Kliniki - Politechnika Wrocławska,3
3,PL. GRUNWALDZKI,5
4,most Grunwaldzki,6
5,Urząd Wojewódzki (Impart),8
6,pl. Wróblewskiego,10
7,GALERIA DOMINIKAŃSKA,13
8,Wzgórze Partyzantów,15
9,DWORZEC GŁÓWNY,17


In [4]:
df['source'] = df['nazwa'].shift(1)
df['target'] = df['nazwa']
df['delta_time'] = df['czas'] - df['czas'].shift(1)
df.dropna(inplace=True)
df = df.drop(['nazwa','czas'], axis=1)


In [70]:
nazwa_lini = 16
df['nazwa_lini'] = nazwa_lini
df

,source,target,delta_time,nazwa_lini
1,ZOO,Hala Stulecia,1.0,16
2,Hala Stulecia,Kliniki - Politechnika Wrocławska,2.0,16
3,Kliniki - Politechnika Wrocławska,PL. GRUNWALDZKI,2.0,16
4,PL. GRUNWALDZKI,most Grunwaldzki,1.0,16
5,most Grunwaldzki,Urząd Wojewódzki (Impart),2.0,16
6,Urząd Wojewódzki (Impart),pl. Wróblewskiego,2.0,16
7,pl. Wróblewskiego,GALERIA DOMINIKAŃSKA,3.0,16
8,GALERIA DOMINIKAŃSKA,Wzgórze Partyzantów,2.0,16
9,Wzgórze Partyzantów,DWORZEC GŁÓWNY,2.0,16
10,DWORZEC GŁÓWNY,DWORZEC GŁÓWNY,2.0,16


In [63]:
G = nx.MultiDiGraph()

for i in range(len(df)-1):
    G.add_edge(df['nazwa'][i], df['nazwa'][i+1])
print(G)
# wyświetl informacje o grafie
#print(nx.info(G))

KeyError: 'nazwa'